# Find the variable names and values that are saved in a checkpoint

In [1]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CAM/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
from tensorflow.python import pywrap_tensorflow
import pickle
import numpy as np
# source: https://stackoverflow.com/a/41917296/9815299

In [3]:
#checkpoint_path = os.path.join(model_dir, "model.ckpt")
checkpoint_path = "/content/drive/My Drive/CAM/models/na4GradCAM/model-24"
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()

# If extract_weight for CAM processing: CAM = True
# Else, in case of Grad-CAM: CAM = False
CAM = False
new_weights = {}
for key in var_to_shape_map:
#     print("tensor_name: ", key)
#     print(reader.get_tensor(key)) # Remove this is you want to print only variable name
    
    layer_name, remainder = key.split('/')
    print('name', layer_name, 'remainder', remainder)
    if layer_name not in new_weights:
        if 'b' in remainder:
            new_weights[layer_name] = [0]
            # append the new number to the existing array at this slot
            new_weights[layer_name].append(reader.get_tensor(key))
        else:
            # create a new array in this slot
            if CAM:
              new_weights[layer_name] = [np.transpose(reader.get_tensor(key))]
            else:
              new_weights[layer_name] = [reader.get_tensor(key)]

    else:
        if 'b' in remainder:
            # append the new number to the existing array at this slot
            new_weights[layer_name].append(reader.get_tensor(key))
        else:
            # create a new array in this slot
            if CAM:
              new_weights[layer_name][0] = np.transpose(reader.get_tensor(key))
            else:
              new_weights[layer_name][0] = reader.get_tensor(key)
#new_weights

('name', 'conv3_2', 'remainder', 'b')
('name', 'conv3_3', 'remainder', 'b')
('name', 'conv2_1', 'remainder', 'W')
('name', 'fc8', 'remainder', 'b')
('name', 'GAP', 'remainder', 'W')
('name', 'conv6', 'remainder', 'b')
('name', 'conv4_3', 'remainder', 'W')
('name', 'conv3_1', 'remainder', 'b')
('name', 'conv3_1', 'remainder', 'W')
('name', 'conv6', 'remainder', 'W')
('name', 'conv4_3', 'remainder', 'b')
('name', 'conv3_2', 'remainder', 'W')
('name', 'fc8', 'remainder', 'W')
('name', 'conv3_3', 'remainder', 'W')
('name', 'conv2_1', 'remainder', 'b')
('name', 'conv2_2', 'remainder', 'W')
('name', 'conv5_2', 'remainder', 'b')
('name', 'fc6', 'remainder', 'b')
('name', 'conv1_1', 'remainder', 'W')
('name', 'conv5_1', 'remainder', 'W')
('name', 'fc7', 'remainder', 'b')
('name', 'conv5_3', 'remainder', 'b')
('name', 'conv4_2', 'remainder', 'b')
('name', 'conv4_1', 'remainder', 'b')
('name', 'conv1_2', 'remainder', 'b')
('name', 'conv4_2', 'remainder', 'W')
('name', 'conv4_1', 'remainder', 'W'

In [0]:
# Save: dict 2 pickle
#https://stackoverflow.com/a/11218504/9815299
with open('../data/new_na_GradCAM.pickle', 'wb') as handle:
    pickle.dump(new_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# Save: dict to npy
np.save("/content/drive/My Drive/CAM/data/new_na_GradCAM.npy", new_weights)

In [6]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.0MB 1.9MB/s 
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4


In [7]:
# compare new weight to vgg_weight
with open('../data/new_na_CAM.pickle', 'r') as f:
    x = pickle.load(f)
# with open('../data/caffe_layers_value.pickle', 'r') as f:
#     y = pickle.load(f)
y = np.load("/content/drive/My Drive/Grad-CAM-tensorflow/model/vgg16.npy", encoding="latin1", allow_pickle=True).item()

# compare 2 dicts
def compare(first, second):
    sharedKeys = set(first.keys()).intersection(second.keys())
    print(sharedKeys)
    for key in sharedKeys:
        print'layer:',key
        if first[key][0].shape != second[key][0].shape:
            print 'Key:', key, 'W1:',first[key][0].shape,' W2:',second[key][0].shape
            print 'W1_transpose:', first[key][0].shape
        else:
            print 'W1:',first[key][0].shape, 'W2:',second[key][0].shape
            
        if  first[key][1].shape != second[key][1].shape:
            print 'Key:', key, 'b1:',first[key][1].shape,' b2:',second[key][1].shape
            print 'b1_transpose', first[key][1].shape
        else:
            print 'b1:',first[key][0].shape, 'b2',second[key][0].shape
compare(x,y)

set(['conv5_1', 'fc6', 'conv5_3', 'conv5_2', 'fc8', 'fc7', 'conv4_1', 'conv4_2', 'conv4_3', 'conv3_3', 'conv3_2', 'conv3_1', 'conv1_1', 'conv1_2', 'conv2_2', 'conv2_1'])
layer: conv5_1
Key: conv5_1 W1: (512, 512, 3, 3)  W2: (3, 3, 512, 512)
W1_transpose: (512, 512, 3, 3)
b1: (512, 512, 3, 3) b2 (3, 3, 512, 512)
layer: fc6
Key: fc6 W1: (4096, 25088)  W2: (25088, 4096)
W1_transpose: (4096, 25088)
b1: (4096, 25088) b2 (25088, 4096)
layer: conv5_3
Key: conv5_3 W1: (512, 512, 3, 3)  W2: (3, 3, 512, 512)
W1_transpose: (512, 512, 3, 3)
b1: (512, 512, 3, 3) b2 (3, 3, 512, 512)
layer: conv5_2
Key: conv5_2 W1: (512, 512, 3, 3)  W2: (3, 3, 512, 512)
W1_transpose: (512, 512, 3, 3)
b1: (512, 512, 3, 3) b2 (3, 3, 512, 512)
layer: fc8
Key: fc8 W1: (1000, 4096)  W2: (4096, 1000)
W1_transpose: (1000, 4096)
b1: (1000, 4096) b2 (4096, 1000)
layer: fc7
W1: (4096, 4096) W2: (4096, 4096)
b1: (4096, 4096) b2 (4096, 4096)
layer: conv4_1
Key: conv4_1 W1: (512, 256, 3, 3)  W2: (3, 3, 256, 512)
W1_transpose: (51

In [9]:
np.shape(x['conv6'][0])

(1024, 512, 3, 3)

In [5]:
def npy2pickle():
  y = np.load("/content/drive/My Drive/Grad-CAM-tensorflow/model/vgg16.npy", encoding="latin1",allow_pickle=False).item()

  with open("/content/drive/My Drive/Grad-CAM-tensorflow/model/vgg16.pickle",'wb') as f: pickle.dump(y, f)

  with open("/content/drive/My Drive/Grad-CAM-tensorflow/model/vgg16.pickle",'rb') as f: y_new = pickle.load(f)

ValueError: ignored

In [13]:
y.keys()

['conv5_1',
 'fc6',
 'conv5_3',
 'fc7',
 'fc8',
 'conv5_2',
 'conv4_1',
 'conv4_2',
 'conv4_3',
 'conv3_3',
 'conv3_2',
 'conv3_1',
 'conv1_1',
 'conv1_2',
 'conv2_2',
 'conv2_1']